In [1]:
from download_manager import DownloadManager, GFSForecastMetadata
import pandas as pd
import xarray as xr
import pygrib
from pathlib import Path
import cfgrib
import os
import re
from download_scripts.GFS.gfs_download_manager import GFSForecastDownloadManager
from datetime import datetime
from nimbus.common.io import run_command
from download_scripts.GFS.config import VARIABLES, REGION_BOUND

In [ ]:
# class GFSForecastDownloadManager(DownloadManager):
#     def __init__(
#             self, 
#             base_dir,
#             buffer_dir,
#             forecast_horizon_hours,
#             issue_times,
#             variables,
#             region_bound,
#             log_filename,
#         ):
#         super().__init__(base_dir=base_dir, buffer_dir=buffer_dir, db_field_dtypes=GFSForecastMetadata.dtypes, log_filename=log_filename)
#         self.variables = variables
#         self.region_bound = region_bound
#         self.forecast_horizon_hours = forecast_horizon_hours
#         self.issue_times = issue_times

#     def callback(self, source_filename, buffer_filename, local_filename):
#         def parse_source_filename(url):
#             # E.g. "https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20220626/00/atmos/gfs.t00z.pgrb2.0p25.f024"
            
#             d = re.search(r'\.(\d{8})/', url).group(1)    # look for . followed by 8 digits
#             h = re.search(r'/(\d{2})/', url).group(1)     # look for two digits surrounded by two /'s
#             f = int(re.search(r'\.f(\d{3})', url).group(1)) # look for . followed by 3 digits
#             t0 = datetime.strptime(d + h, "%Y%m%d%H")
#             return t0, f

#         temp_filename = f"{buffer_filename}.partial.grib2"
#         # filter_key = f":({'|'.join(PRESSURE_VARIABLE_KEYS)}):({'|'.join(map(str, PLEVELS))}) mb:|" + '|'.join(SURFACE_VARIABLES)
#         filter_key = '|'.join(self.variables)
#         Path(local_filename).parent.mkdir(parents=True, exist_ok=True)
#         compress_command = f"wgrib2 {buffer_filename} -match '{filter_key}' -small_grib {self.region_bound} {temp_filename}"
#         conversion_command = f"wgrib2 {temp_filename} -netcdf {local_filename}"

#         run_command(compress_command)
#         run_command(conversion_command)

#         Path(buffer_filename).unlink()
#         Path(temp_filename).unlink()

#         utc_issue_timestamp, forecast_horizon = parse_source_filename(source_filename)
#         return GFSForecastMetadata(
#                 source_filename=source_filename,
#                 local_filename=local_filename,
#                 size=os.stat(local_filename).st_size,
#                 last_modified=pd.Timestamp.now('UTC'),
#                 utc_issue_timestamp=utc_issue_timestamp,
#                 forecast_horizon=forecast_horizon
#             )

#     def _calculate_filenames_for_range(self, start: pd.Timestamp, end: pd.Timestamp):
#         utc_issue_timestamps = pd.date_range(start.floor('D'), end.ceil('D'), freq='6h')
#         utc_issue_timestamps = utc_issue_timestamps[
#             utc_issue_timestamps.hour.isin(self.issue_times)
#             & (utc_issue_timestamps >= start)
#             & (utc_issue_timestamps <= end)]

#         source_filenames, buffer_filenames, local_filenames, db_filenames = [], [], [], []
#         for utc_issue_timestamp in utc_issue_timestamps:
#             for forecast_horizon_hour in self.forecast_horizon_hours:
#                 str_format = utc_issue_timestamp.strftime(f"gfs.%Y%m%d/%H/atmos/gfs.t%Hz.pgrb2.0p25.f{forecast_horizon_hour:03d}")
#                 source_filenames.append(f"https://noaa-gfs-bdp-pds.s3.amazonaws.com/{str_format}")
#                 buffer_filenames.append(str_format)
#                 local_filenames.append(utc_issue_timestamp.strftime(f"%Y_%m/%d/%Y_%m_%d_%H:00_f{forecast_horizon_hour:03d}.nc"))
#                 db_filenames.append(utc_issue_timestamp.strftime("%Y_%m.db"))

#         return source_filenames, buffer_filenames, local_filenames, db_filenames

#     def cleanup(self):
#         root = Path(self.buffer_dir)
#         for path in sorted(root.rglob('*'), key=lambda p: len(p.parts), reverse=True): # deepest directories first
#             if path.is_dir() and not any(path.iterdir()): # remove if empty (Path.rmdir() cannot remove non-empty dir anyways, so it is safe.)
#                 path.rmdir()

In [2]:
dm = GFSForecastDownloadManager(base_dir=f"/mnt/lustre/koa/koastore/sadow_group/shared/wrf_hawaii/raw_data/gfs/", 
                                buffer_dir='/mnt/lustre/koa/scratch/yusukemh/wrf-diffusion/gfs_buffer',
                                log_filename='stdout', forecast_horizon_hours=range(10), issue_times=[0,6,12,18],
                                variables=VARIABLES, region_bound=REGION_BOUND)

In [15]:
start = pd.Timestamp('2021-04-01 00:00:00')
end = pd.Timestamp('2021-04-01 01:00:00')

In [16]:
source_filenames, buffer_filenames, local_filenames, database_filenames = dm.calculate_filenames_for_range(start=start, end=end)

In [17]:
len(source_filenames)

10

In [ ]:
array_parser = ArrayParser(year=range(2002, 2026), month=range(1,13))

In [18]:
dm.download_files(mode='range', start=start, end=end)

02/24 13:00 HST [INFO]:</home/yusukemh/github/DownloadManager/download_manager/download_manager.py:93> in download_files, Calculating filenames
02/24 13:00 HST [INFO]:</home/yusukemh/github/DownloadManager/download_manager/download_manager.py:107> in download_files, Checking database for already existing files.
02/24 13:00 HST [INFO]:</home/yusukemh/github/DownloadManager/download_manager/download_manager.py:119> in download_files, Found 10 new files.
02/24 13:01 HST [INFO]:</home/yusukemh/github/DownloadManager/download_manager/download_manager.py:153> in download_files, Completed downloading.


In [13]:
dm.cleanup()

In [10]:
df = dm.export_databases()

In [12]:
df[0].sort_values('last_modified').sort_values('forecast_horizon')

,product,datatype,source_filename,local_filename,size,last_modified,forecast_horizon,utc_issue_timestamp
1,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181528,2026-02-24 22:25:48.530332+00:00,0,2023-01-03 00:00:00
0,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:25:48.388916+00:00,1,2023-01-03 00:00:00
3,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:25:49.448030+00:00,2,2023-01-03 00:00:00
5,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:26:19.979599+00:00,3,2023-01-03 00:00:00
2,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:25:49.323010+00:00,4,2023-01-03 00:00:00
4,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:25:52.311953+00:00,5,2023-01-03 00:00:00
7,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:26:57.304111+00:00,6,2023-01-03 00:00:00
6,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:26:56.412057+00:00,7,2023-01-03 00:00:00
16,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:27:36.171912+00:00,8,2023-01-03 00:00:00
17,GFS,forecast,https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs....,/mnt/lustre/koa/koastore/sadow_group/shared/wr...,181540,2026-02-24 22:27:47.674642+00:00,9,2023-01-03 00:00:00
